In [ ]:
# default_exp core.rq_wantlist

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *
import json
import responses
from pathlib import Path
from discogspy.core import *

In [ ]:
#hide
with open(Path("../config/example_user_info.json"), "r") as user_config_file:
    config = json.load(user_config_file)
    example_user = UserWithUserTokenBasedAuthentication(user_token=config["user_token"],
                                                        user_agent=config["user_agent"])

# Core Wantlist Requests

> Core.rq_wantlist is a collection of function wrappers around the Discogs api for getting user wantlist information.

In [ ]:
#export
import requests
from typing import Union
from discogspy.core import *

In [ ]:
#export


def get_user_wantlist(user: Union[UserWithoutAuthentication,
                                  UserWithUserTokenBasedAuthentication],
                      username: str,
                      page: Union[int, None] = None,
                      per_page: Union[int, None] = None,
                      ) -> requests.models.Response:
    """
    Get a list of releases from the wantlist of a given user. 
    
    Note: If the wantlist has been made private by its owner, 
          you must be authenticated as the owner to view it.
    
    No user Authentication needed. 
    """
    
    url = f"{USERS_URL}/{username}/wants"
    headers = user.headers
    params = user.params
    
    if page:
        params["page"] = max(1, page)
    if per_page:
        params["per_page"] = max(1, per_page)
    
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_user_wantlist
get_user_wantlist(user=example_user, username="discogspy", page=1, per_page=3)

In [ ]:
#hide

@responses.activate
def test_get_user_wantlist___authentication_none():
    test_user = UserWithoutAuthentication()
    
    target_url = f"{USERS_URL}/user_name/wants?page=1&per_page=3"
    
    result_content = {"TestWantlist": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_user_wantlist(user=test_user, username="user_name", page=1, per_page=3)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_user_wantlist___authentication_none()

@responses.activate
def test_get_user_wantlist___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{USERS_URL}/user_name/wants?token=test_user_token&page=1&per_page=3"
        
    result_content = {"TestWantlist": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_user_wantlist(user=test_user, username="user_name", page=1, per_page=3)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_user_wantlist___authentication_user_token()

In [ ]:
#hide

from nbdev.export import *
notebook2script()